In [ ]:
!pip install openai
!pip install langchain
!pip install -U langchain-community
!pip install pypdf

In [2]:
import os
import openai
import sys

In [ ]:
from langchain.document_loaders import PyPDFLoader

loaders = [
    PyPDFLoader("/content/One Up On Wall Street by Peter Lynch ( PDFDrive ).pdf"),
    PyPDFLoader("/content/The Intelligent Investor ( PDFDrive ).pdf")
]

docs = []

for loader in loaders:
  try:
    docs.extend(loader.load())
  except TypeError as e:
    print(f"Error loading document: {e}")

In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1500,
    chunk_overlap = 150
)

In [9]:
splits = text_splitter.split_documents(docs)

In [10]:
len(splits)

519

### Embeddings

In [ ]:
!pip install tiktoken

In [47]:
import getpass
import os
import numpy as np

#if not os.getenv("OPENAI_API_KEY"):
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

Enter your OpenAI API key: ··········


In [48]:
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()

In [49]:
sentence1 = "i like dogs"
sentence2 = "i like canines"
sentence3 = "the weather is ugly outside"

In [ ]:
embedding1 = embedding.embed_query(sentence1)
embedding2 = embedding.embed_query(sentence2)
embedding3 = embedding.embed_query(sentence3)

In [ ]:
# Calculate the similarity between the embeddings
# The higher the value, the higher the similarity between the embeddings
np.dot(embedding1, embedding2)
np.dot(embedding1, embedding3)
np.dot(embedding2, embedding3)

### Vectorstores

In [51]:
!pip install chromadb

In [52]:
from langchain.vectorstores import Chroma

In [53]:
persist_directory = "content/chroma/"

In [54]:
!rm -rf ./docs/chroma  # remove old database files if any

In [ ]:
vectorbd = Chroma.from_documents(
    documents = splits,
    embedding = embedding,
    persist_directory = persist_directory
)

In [ ]:
print(vectordb._collection.count())

### Similarity Search

In [ ]:
question = "is there an email i can ask for help"

In [ ]:
# k=3, return 3 chuncks with the most similarity
docs = vectordb.similarity_search(question,k=3)

In [ ]:
len(docs)

In [ ]:
docs[0].page_content

In [ ]:
for doc in docs:
    print(doc.metadata)

'''
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 0}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 14}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture02.pdf', 'page': 0}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture03.pdf', 'page': 6}
{'source': 'docs/cs229_lectures/MachineLearning-Lecture01.pdf', 'page': 8}
'''